In [55]:
import pandas as pd
import numpy as np

# Load the datasets
file1_path = 'educ_data.csv'
file2_path = 'gini_data.csv'
file3_path = 'unemp_data.csv'

educ_df = pd.read_csv(file1_path)
gini_df = pd.read_csv(file2_path)
unemp_df = pd.read_csv(file3_path)

In [56]:
# Clean unemp_df
id_vars = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']  
year_columns = unemp_df.columns[4:]  # All the year columns, starting from the 5th column onward

# Melt the dataframe to bring year columns into rows
unemp_df = pd.melt(unemp_df, id_vars=id_vars, value_vars=year_columns, var_name='Year', value_name='Unemp %')

# Drop rows with no data
unemp_df = unemp_df.dropna(subset=['Unemp %'])

# Convert 'Year' to a numerical type if needed (strip out the text around it, like 'YR1960' to just '1960')
unemp_df['Year'] = unemp_df['Year'].str.extract('(\d+)').astype(int)

# Clean educ_df
id_vars = ['Country Name', 'Country Code', 'Series Name', 'Series Code']  
year_columns = educ_df.columns[4:]  # All the year columns, starting from the 5th column onward

# Melt the dataframe to bring year columns into rows
educ_df = pd.melt(educ_df, id_vars=id_vars, value_vars=year_columns, var_name='Year', value_name='Average years of schooling')

# Drop rows with no data
educ_df = educ_df[educ_df['Average years of schooling'] != '..']

# Convert 'Year' to a numerical type if needed (strip out the text around it, like 'YR1960' to just '1960')
educ_df['Year'] = educ_df['Year'].str.extract('(\d+)').astype(int)

# Clean gini_df

gini_df["iso"] = gini_df["iso"].transform(lambda x: x.upper())


In [59]:
merged_df = pd.merge(gini_df,unemp_df,left_on=["iso","year"],right_on=["Country Code","Year"])
merged_df = pd.merge(merged_df,educ_df,left_on=["iso","year"],right_on=["Country Code","Year"])

columns_req = ['country', 'iso', 'year', 'gini_reported',
       'gdp_ppp_pc_usd2011', 'population',
       'gini_recalculated','Unemp %',
       'Series Name','Average years of schooling']

merged_df = merged_df[columns_req]
merged_df = merged_df[merged_df['Series Name'].str.contains('both sexes', case=False)]

# Step 2: Drop duplicate rows
merged_df = merged_df.drop_duplicates().reset_index(drop=True)
merged_df = merged_df.set_index(["country","year"])

In [60]:
merged_df.head()

iso gini_reported  gdp_ppp_pc_usd2011    population   
country year                                                        
Angola  2014  AGO     0.4445348         6283.000000  2.254955e+07  \
Albania 2001  ALB   0.329893478         5845.833333  3.121602e+06   
        2008  ALB   0.319450431         8986.000000  3.002683e+06   
        2011  ALB   0.312414251         9987.250000  2.936239e+06   
        2012  ALB   0.310068857        10321.000000  2.914091e+06   

             gini_recalculated  Unemp %   
country year                              
Angola  2014         0.4503197   16.401  \
Albania 2001       0.330970731   18.570   
        2008       0.319450432   13.060   
        2011       0.314103232   13.481   
        2012       0.310068857   13.376   

                                                    Series Name   
country year                                                      
Angola  2014  UIS: Mean years of schooling (ISCED 1 or highe...  \
Albania 2001  UIS: Mean years of schooling (ISCED 1 or highe...   
        2008  UIS: Mean years of schooling (ISCED 1 or highe...   
        2011  UIS: Mean years of schooling (ISCED 1 or highe...   
        2012  UIS: Mean years of schooling (ISCED 1 or highe...   

             Average years of schooling  
country year                             
Angola  2014                    3.99596  
Albania 2001                    8.74141  
        2008                    9.16803  
        2011                    9.90485  
        2012                   10.14573